In [1]:
from pymilvus import connections, Collection, utility
from pymilvus import utility
import google.generativeai as genai
from local_settings import API_KEY,uri,token
from pymilvus import MilvusClient
from helper_func import create_prompt, process_data

connections.connect(uri=uri,token=token)
milvus_client = MilvusClient(uri=uri,token=token)

collection_name = "my_rag_collection2"

In [2]:

collection = Collection(collection_name)
collection.num_entities 

21326

In [3]:
utility.list_collections()

['my_rag_collection2']

In [4]:
res = collection.query(expr="", output_fields=["count(*)"])
print(res)

[{'count(*)': 21326}]


In [5]:
embedding_model = 'models/text-embedding-004'
genai.configure(api_key=API_KEY)
def embed_fn(title, text):
  return genai.embed_content(model=embedding_model,
                             content=text,
                             task_type="retrieval_document",
                             title=title)["embedding"]

In [6]:
question = "What Are Tax Sheltered Annuities?"

search_res = milvus_client.search(
    collection_name=collection_name,
    data=[
        embed_fn(question,question)
    ],  # Use the `emb_text` function to convert the question to an embedding vector
    limit=7,  # Return top 3 results
    search_params={"metric_type": "COSINE",  "params": {"level": 2}},  # Inner product distance
    output_fields=["text"],  # Return the text field
)

In [7]:
retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
retrieved_lines_with_distances

[(['What Are Tax Sheltered Annuities? ',
   'Qualified annuities generally refer to tax deferred annuities is a qualified plan like ab ERISA -LRB- Employee Retirement Income Security Act of 1974 -LRB- qualified plan such as a 410 -LRB- k -RRB- . By inserting tax deferred annuities into a qualified plan contribution re-characterizes the basis as taxable during distributions , i.e. the entire annuity distributions are taxable as ordinary income . '],
  0.8431991934776306),
 (['What Are Tax Sheltered Annuities? ',
   'An employer can establish a plan in which contributions are deducted from the employees reported income and the money is invested in an annuity . The money placed in the annuity and the interest that it earns are not subject to current income tax . The benefits , when withdrawn are taxable as ordinary income . The best known Tax Sheltered Annuity is allowed under section 403 -LRB- b -RRB- of the Internal Revenue Code . '],
  0.8355906009674072),
 (['What Are Tax Sheltered An

In [8]:

querry_data= process_data(retrieved_lines_with_distances)
print(querry_data)

Q: What Are Tax Sheltered Annuities?
A: Qualified annuities generally refer to tax deferred annuities is a qualified plan like ab ERISA -LRB- Employee Retirement Income Security Act of 1974 -LRB- qualified plan such as a 410 -LRB- k -RRB- . By inserting tax deferred annuities into a qualified plan contribution re-characterizes the basis as taxable during distributions , i.e. the entire annuity distributions are taxable as ordinary income .

Q: What Are Tax Sheltered Annuities?
A: An employer can establish a plan in which contributions are deducted from the employees reported income and the money is invested in an annuity . The money placed in the annuity and the interest that it earns are not subject to current income tax . The benefits , when withdrawn are taxable as ordinary income . The best known Tax Sheltered Annuity is allowed under section 403 -LRB- b -RRB- of the Internal Revenue Code .

Q: What Are Tax Sheltered Annuities?
A: Tax sheltered annuities -LRB- TSA 's -RRB- are a ty

In [14]:
%%time
testing_question="What Are Tax Sheltered Annuities?"
prompt=create_prompt(querry_data,testing_question)
print(prompt)


Task: Your are a Insurance assistance, your job is based on a conversation of a costumer and a insurance expert, 
answer insurance-related questions accurately and provide some short explanation.
Instructions:
Read the context carefully.
Consider key points, concepts, and details within the document.
Generate questions that seek information or clarification about the document's content.
Ensure that the questions are grammatically correct and understandable.
Give the answer directly, do not repeat the question or add any unrelated text
If the user talk to you casually, you can rely back
If the question is blank, tell the user to type the question in.
If the question is not related to the context or there is no answer in the context, tell the users the question is not related to insurance or there is no answer in your database
Context:
Q: What Are Tax Sheltered Annuities?
A: Qualified annuities generally refer to tax deferred annuities is a qualified plan like ab ERISA -LRB- Employee Re

In [15]:
%%time
LLM_model = genai.GenerativeModel('gemini-pro')
response = LLM_model.generate_content(prompt)

CPU times: total: 0 ns
Wall time: 4.11 s


In [16]:
print( response.text.replace("JSON", "").replace("```",""))

Tax sheltered annuities -LRB- TSA 's -RRB- are a type of retirement plan that allows employees of eligible public educational organizations or tax exempt organizations to contribute part of their pre-taxed salary to the plan with tax deferred growth . Employers are also allowed to contribute to a tax sheltered annuity for the employee . Authorized by IRS Section 403 -LRB- b -RRB- , TSA 's are also called 403 -LRB- b -RRB- plans .
